In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_groq import ChatGroq

model=ChatGroq(api_key=os.getenv("GROQ_API_KEY") , model="llama-3.1-8b-instant")


#### - Simple Model invoking

In [ ]:
response=model.invoke("What is Maths? . explain math in one sentence")
print(response.content)

### -Streaming O/P


In [ ]:
for chunk in model.stream("generate 1000 word paragraph about Machine Learning"):
    print(chunk.content  , end="", flush=True)

## Looping through list of questions... (batching)

In [ ]:
# you can send multiple prompts as input in one single request...
requests= [
    "who is Quaid-e-Azam",
    "who is Allama-Iqbal",
    "who is Rock"
]

print("Processing your input :) ")

multiple_responses=model.batch(
    requests,
    config={"max_concurrency":1}
)

print(multiple_responses)


### Using prompt tempelates

In [ ]:
#-importing chat prompt tempelate.
from langchain_core.prompts import ChatPromptTemplate
#-define a Prompt Template with vairable slot
prompt=ChatPromptTemplate.from_template(
    "Explain {topic} to a 5 year old kid."
)
gen_response=prompt.invoke({"topic":"Civil Engineering"})
print("Human Message ----->",gen_response) #----- topic civil engeering is automatically specifed to variable topic .

gen2=model.invoke(gen_response)
print(gen2.content)

#### Output Parser

#

In [ ]:
from langchain_core.output_parsers import StrOutputParser
#-create parser object
parser=StrOutputParser()
# before invoking the parser we must have ai generated respones
ai_response=model.invoke("Who is PM of Pakistan as well as Punjab?")
print(ai_response)

# now we get the use of parser
parser.invoke(ai_response)

# LCEL Chains


In [ ]:
# lets create our chain
chain_prompt=ChatPromptTemplate.from_template(
    """Respond to the user query
    Query = {query}
    """
)
invoked=chain_prompt.invoke({"query":"enlist 10 popular tv shows served on PTV HOME"})

chain = chain_prompt | model | parser

## Creating custom tools
LLM are smarter they know better which tool to choose the can decide wisely

In [ ]:
from langchain_core.tools import tool

@tool
#must add doc string
def multiply_tool(first_num:int,second_num:int):
    """This function multiply or find product out of two numbers"""
    return first_num*second_num

### Binding tools with LLM

In [ ]:
tools=[multiply_tool]
model_with_tools=model.bind_tools(tools=tools)

In [ ]:
response2=model_with_tools.invoke("Multiply 2 with 3")
print(response2.tool_calls)

# PreBuilt TooLS

In [ ]:
from langchain_community.tools import TavilySearchResults
search_tool=TavilySearchResults(max_results=5)
tools = [search_tool]
result=search_tool.invoke("What's the current rate of $ in Pakistan ?")
print(result)

## - Creating our first agent with toooooooolsss.

In [ ]:
#from langgraph.prebuilt import create_react_agent ---> It is depricated in latest version of langchain + langgraph
from langchain.agents import create_agent

my_agent=create_agent(
    model=model,
    tools=tools
)
print("Agent is created successfully...")

In [ ]:
query = "who is founder of Apple"
agent_response=my_agent.invoke({"messages":[("user",query)]})
agent_response["messages"][-1].content

In [ ]:
query = "search for best hotels near me"
agent_response=my_agent.invoke({"messages":[("user",query)]})
agent_response["messages"][-1].content

## Creating the state schema -----pass only the data that is required

In [ ]:
# we use pydantic basemodel to get structured output

from pydantic import BaseModel,Field

"""Suppose you wants to upload a post onto facebook
   what all things you require a title
   a photo (optional) 
   --photo can't be a GIF
   """

class FacebookPost(BaseModel):
    post_title:str=Field(description="Your Post title")
    post_tags:list[str]=Field(description="Catchy___Tags")
    

In [ ]:
model

In [ ]:
structure=model.with_structured_output(FacebookPost)

In [ ]:
result_of_strucured_output=structure.invoke("create a facebook post related to discovery of jungle")

In [ ]:
print("TITLE------>",result_of_strucured_output.post_title)
print("TAGS------->",result_of_strucured_output.post_tags)

#### _____________________________________________________________________________

## Adding memory to agents

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langchain.agents import create_agent
import time
print("______________Creating memory slot in your space________________")
time.sleep(2)
print("Memory is beeeeeN   Added to your agent")
#-create memory object

memory=MemorySaver()

#-Binding memory to the agent

agent2=create_agent(
    model,
    tools,
    checkpointer=memory
)

## __________________________________________________
#### Managing Sessions

In [ ]:
#-one single chat has 1 session and a session id which has memory thorughout the chat
config1 ={"configurable": {
    "thread_id":"chat_1"
}}

agent2.invoke({"messages":"My name is Zain"} , config=config1)

In [ ]:
r=agent2.invoke({"messages" : "What is my name"},config=config1)
r["messages"][-1].content

# _________________________________________________________
### Text to vector

In [ ]:
#----Data ingestion
from langchain_community.document_loaders import TextLoader

loader=TextLoader(file_path="text_file.txt")
docs=loader.load()
docs

In [ ]:
# splitting text
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter=RecursiveCharacterTextSplitter()
final_doc=splitter.split_documents(docs)


In [ ]:
#-converting into embeddings
from langchain_ollama.embeddings import OllamaEmbeddings

embedding_model=OllamaEmbeddings(model="all-minilm")


In [ ]:
#-creating a vector store FAISS
from langchain_community.vectorstores import FAISS
from langchain.chat_models.
vector_store=FAISS.from_documents(
    documents=final_doc, #----docs are our original data source
    embedding=embedding_model
)

retriever=vector_store.as_retriever()

In [ ]:
retriever.invoke("Engineering")

## _________________________________________________________________________________________
### Creating RAG Chain.

In [ ]:
'''
Unable to import "create_retrieval_chain" thats why this session is Closed
'''

In [ ]:
"""
There is also a concept of HITL (Human in the loop) we will cover that in langraph module.
"""

### Tracking of our projects

In [ ]:
# Use this in your environment varaibles
LANGCHAIN_TRACING_V2=True
LANGCHAIN_API_KEY="your_api_key"
LANGCHAIN_PROJECT="Project-name"